# 第6章 广度优先搜索

本章将介绍图。首先，我将说说什么是图（它们不涉及X轴和Y轴），再介绍第一种图算法——广度优先搜索（breadth-first search，BFS）。
广度优先搜索让你能够找出两样东西之间的最短距离，不过最短距离的含义有很多！使用广度优先搜索可以：

- 编写国际跳棋AI，计算最少走多少步就可获胜；
- 编写拼写检查器，计算最少编辑多少个地方就可将错拼的单词改成正确的单词，如将READED改为READER需要编辑一个地方；
- 根据你的人际关系网络找到关系最近的医生。

在我所知道的算法中，图算法应该是最有用的。请务必仔细阅读接下来的几章，这些算法你将经常用到。

## 6.1 图简介

这种问题被称为最短路径问题（shorterst-path problem）。你经常要找出最短路径，这可能是前往朋友家的最短路径，也可能是国际象棋中把对方将死的最少步数。解决最短路径问题的算法被称为广度优先搜索。

要确定如何从双子峰前往金门大桥，需要两个步骤。

- (1) 使用图来建立问题模型。
- (2) 使用广度优先搜索解决问题。

## 6.2 图是什么

图由节点（node）和边（edge）组成。图由节点和边组成。一个节点可能与众多节点直接相连，这些节点被称为邻居。图用于模拟不同的东西是如何相连的。

## 6.3 广度优先搜索

第1章介绍了一种查找算法——二分查找。广度优先搜索是一种用于图的查找算法，可帮助
回答两类问题。

- 第一类问题：从节点A出发，有前往节点B的路径吗？
- 第二类问题：从节点A出发，前往节点B的哪条路径最短？

### 6.3.1 查找最短路径

你按顺序依次检查名单中的每个人，看看他是否是芒果销售商。这将先在一度关系中查找，再在二度关系中查找，因此找到的是关系最近的芒果销售商。

广度优先搜索不仅查找从A到B的路径，而且找到的是最短的路径。

只有按添加顺序查找时，才能实现这样的目的。因此，你需要按添加顺序进行检查。有一个可实现这种目的的数据结构，那就是队列（queue）。

### 6.3.2 队列

队列类似于栈，你不能随机地访问队列中的元素。队列只支持两种操作：入队和出队。

如果你将两个元素加入队列，先加入的元素将在后加入的元素之前出队。因此，你可使用队列来表示查找名单！这样，先加入的人将先出队并先被检查。**队列是一种先进先出（First In First Out，FIFO）的数据结构，而栈是一种后进先出（Last InFirst Out，LIFO）的数据结构。**

## 6.4 实现图

首先，需要使用代码来实现图。图由多个节点组成。每个节点都与邻近节点相连，如果表示类似于“你→Bob”这样的关系呢？

好在你知道的一种结构让你能够表示这种关系，它就是散列表！记住，散列表让你能够将键映射到值。在这里，你要将节点映射到其所有邻居。

表示这种映射关系的Python代码如下。

In [1]:
graph = {}
graph["you"] = ["alice", "bob", "claire"]

注意，“你”被映射到了一个数组，因此 graph["you"] 是一个数组，其中包含了“你”的所有邻居。图不过是一系列的节点和边，因此在Python中，只需使用上述代码就可表示一个图。

那像下面这样更大的图呢？

[![eDAU3D.png](https://s2.ax1x.com/2019/08/03/eDAU3D.png)](https://imgchr.com/i/eDAU3D)

表示它的Python代码如下。

In [2]:
graph = {}
graph["you"] = ["alice", "bob", "claire"]
graph["bob"] = ["anuj", "peggy"]
graph["alice"] = ["peggy"]
graph["claire"] = ["thom", "jonny"]
graph["anuj"] = []
graph["peggy"] = []
graph["thom"] = []
graph["jonny"] = []

In [3]:
graph

{'alice': ['peggy'],
 'anuj': [],
 'bob': ['anuj', 'peggy'],
 'claire': ['thom', 'jonny'],
 'jonny': [],
 'peggy': [],
 'thom': [],
 'you': ['alice', 'bob', 'claire']}

顺便问一句：键 — 值对的添加顺序重要吗？换言之，如果你这样编写代码：
```
graph["claire"] = ["thom", "jonny"]
graph["anuj"] = []
```
而不是这样编写代码：
```
graph["anuj"] = []
graph["claire"] = ["thom", "jonny"]
```
对结果有影响吗？

只要回顾一下前一章介绍的内容，你就知道没影响。散列表是无序的，因此添加键 — 值对的顺序无关紧要。

Anuj、Peggy、Thom和Jonny都没有邻居，这是因为虽然有指向他们的箭头，但没有从他们出发指向其他人的箭头。这被称为**有向图（directed graph）**，其中的关系是单向的。因此，Anuj是Bob的邻居，但Bob不是Anuj的邻居。

**无向图（undirected graph）**没有箭头，直接相连的节点互为邻居。例如，下面两个图是等价的。

![](https://i.loli.net/2019/08/10/gVrBD8h9v3dXw4G.png)

## 6.5 实现算法

工作原理：

![](https://i.loli.net/2019/08/10/tsY8zRuljcVvZD6.png)

首先，创建一个队列。在Python中，可使用函数 deque 来创建一个双端队列。

In [6]:
from collections import deque
search_queue = deque() # 创建查找队列
search_queue += graph["you"] # 将你的邻居都加入到这个队列中

In [7]:
search_queue

deque(['alice', 'bob', 'claire'])

graph["you"] 是一个数组，其中包含你的所有邻居，如 ["alice", "bob","claire"] 。这些邻居都将加入到搜索队列中。

编写函数 person_is_seller ，判断一个人是不是芒果销售商，如下所示。

这个函数检查人的姓名是否以m结尾：如果是，他就是芒果销售商。这种判断方法有点搞笑，

但就这个示例而言是可行的。下面来看看广度优先搜索的执行过程。

In [12]:
def person_is_seller(name):
    return name[-1] == 'm'

In [22]:
from collections import deque

def search(person):
    search_queue = deque() # 创建查找队列
    search_queue += graph[person] # 将你的邻居都加入到这个队列中    
    while search_queue: #只要队列不空
        person = search_queue.popleft() #就取出其中的第一个人并返回给person
        if person_is_seller(person): #检查是否芒果经销商
            print(person + " is a mango seller!")
            return True
        else:
            search_queue += graph[person] #不是芒果销售商。将这个人的朋友都加入搜索队列
    return False #如果到达了这里，就说明队列中没人是芒果销售商

print(search("you"))

thom is a mango seller!
True


这个算法将不断执行，直到满足以下条件之一：

- 找到一位芒果销售商；
- 队列变成空的，这意味着你的人际关系网中没有芒果销售商。

Peggy既是Alice的朋友又是Bob的朋友，因此她将被加入队列两次：一次是在添加Alice的朋
友时，另一次是在添加Bob的朋友时。因此，搜索队列将包含两个Peggy。

但你只需检查Peggy一次，看她是不是芒果销售商。如果你检查两次，就做了无用功。因此，检查完一个人后，应将其标记为已检查，且不再检查他。如果不这样做，就可能会导致无限循环。假设你的人际关系网类似于下面这样。搜索队列将在包含你和包含Peggy之间反复切换。

![](https://i.loli.net/2019/08/10/V2KIx83GHakTSln.png)

检查一个人之前，要确认之前没检查过他，这很重要。为此，你可使用一个列表来记录检查过的人。

In [24]:
def search(name):
    search_queue = deque()
    search_queue += graph[name]
    searched = []
    while search_queue:
        person = search_queue.popleft()
        if not person in searched:
            if person_is_seller(person):
                print(person + " is a mango seller!")
                return True
            else:
                search_queue += graph[person]
                searched.append(person)
    return False
print(search("you"))
print(search("bob"))

thom is a mango seller!
True
False


### 运行时间

如果你在你的整个人际关系网中搜索芒果销售商，就意味着你将沿每条边前行（记住，边是从一个人到另一个人的箭头或连接），因此运行时间至少为O(边数)。

你还使用了一个队列，其中包含要检查的每个人。将一个人添加到队列需要的时间是固定的，即为O(1)，因此对每个人都这样做需要的总时间为O(人数)。所以，广度优先搜索的运行时间为O(人数 + 边数)，这通常写作O(V + E)，其中V为顶点（vertice）数，E为边数。


如果任务A依赖于任务B，在列表中任务A就必须在任务B后面。这被称为拓扑排序，使用它可根据图创建一个有序列表。假设你正在规划一场婚礼，并有一个很大的图，其中充斥着需要做的事情，但却不知道要从哪里开始。这时就可使用拓扑排序来创建一个有序的任务列表。

树是一种特殊的图，其中没有往后指的边。

### 小结

- 广度优先搜索指出是否有从A到B的路径。
- 如果有，广度优先搜索将找出最短路径。
- 面临类似于寻找最短路径的问题时，可尝试使用图来建立模型，再使用广度优先搜索来解决问题。
- 有向图中的边为箭头，箭头的方向指定了关系的方向，例如，rama→adit表示rama欠adit钱。
- 无向图中的边不带箭头，其中的关系是双向的，例如，ross - rachel表示“ross与rachel约会，而rachel也与ross约会”。
- 队列是先进先出（FIFO）的。
- 栈是后进先出（LIFO）的。
- 你需要按加入顺序检查搜索列表中的人，否则找到的就不是最短路径，因此搜索列表必须是队列。
- 对于检查过的人，务必不要再去检查，否则可能导致无限循环。